## Import libraries

In [1]:
import pandas as pd

## Extrair os dados de importação do período 2023-01 até 2023-03

In [2]:
df_exportacao_mun = pd.read_csv('../data/01_raw/EXP_2023_MUN.csv', sep = ';')
df_exportacao = pd.read_csv('../data/01_raw/EXP_2023.csv', sep = ';')

In [3]:
df_exportacao_mun = df_exportacao_mun[(df_exportacao_mun['CO_MES'].isin([1,2,3])) & (df_exportacao_mun['CO_ANO']==2023)]
df_exportacao = df_exportacao[(df_exportacao['CO_MES'].isin([1,2,3])) & (df_exportacao['CO_ANO']==2023)]

## Criar um dataset com os arquivos extraídos que contenham as colunas Município e NCM.

In [4]:
# Converto o código de entrada para string, e preencho com zeros a esquerda
df_exportacao['CO_NCM'] = df_exportacao['CO_NCM'].astype(str).str.zfill(8)
df_exportacao_mun['SH4'] = df_exportacao_mun['SH4'].astype(str).str.zfill(4)
# Crio variável SH4
df_exportacao['SH4'] = df_exportacao['CO_NCM'].str.slice(0,4)
#  40.11.1000 -> 8 dígitos

In [5]:
# Converter novamente para inteiro os códigos SH4 e CO_NCM. Int64 aceita nulos. int não aceita nulos
df_exportacao[['CO_NCM', 'SH4']] = df_exportacao[['CO_NCM', 'SH4']].astype('Int64')
df_exportacao_mun['SH4'] = df_exportacao_mun['SH4'].astype('Int64')

### Left join entre as bases

In [6]:
# df_exportacao_mun_ncm = df_exportacao[['SH4', 'CO_NCM', 'CO_ANO', 'CO_MES', 'CO_PAIS']].merge(df_exportacao_mun[['SH4', 'CO_MUN', 'CO_ANO', 'CO_MES', 'CO_PAIS']],
#                                                                 how = 'left', on = ['SH4', 'CO_ANO', 'CO_MES', 'CO_PAIS'])

In [7]:
df_exportacao_mun_ncm = df_exportacao.merge(df_exportacao_mun, how = 'left', on = ['CO_ANO', 'CO_MES', 'SH4', 'CO_PAIS', 'KG_LIQUIDO', 'VL_FOB'])

In [8]:
df_exportacao_mun_ncm['CO_MUN'] = df_exportacao_mun_ncm['CO_MUN'].astype('Int64')

In [9]:
# Seleciono apenas as colunas que preciso
df_exportacao_mun_ncm = df_exportacao_mun_ncm[['CO_NCM', 'CO_MUN', 'CO_VIA', 'CO_URF']]

## Complementar o dataset com as descrições dos campos utilizando os datasets auxiliares nos links:

In [10]:
df_ncm_class = pd.read_csv('../data/01_raw/NCM.csv', sep = ';', encoding="ISO-8859-1")
df_pais_origem = pd.read_csv('../data/01_raw/PAIS.csv', sep = ';', encoding="ISO-8859-1")
df_municipio  = pd.read_csv('../data/01_raw/UF_MUN.csv', sep = ';', encoding="ISO-8859-1")
df_urf  = pd.read_csv('../data/01_raw/URF.csv', sep = ';', encoding="ISO-8859-1")
df_via  = pd.read_csv('../data/01_raw/VIA.csv', sep = ';')

In [11]:
# df_ncm_class.head(2)
df_ncm_class = df_ncm_class[['CO_NCM', 'NO_NCM_POR', 'NO_NCM_ESP', 'NO_NCM_ING']]
df_exportacao_mun_ncm = df_exportacao_mun_ncm.merge(df_ncm_class, on = 'CO_NCM', how = 'left')

In [12]:
# Renomeei para facilitar no merge
df_municipio = df_municipio.rename(columns={'CO_MUN_GEO':'CO_MUN'})
df_exportacao_mun_ncm = df_exportacao_mun_ncm.merge(df_municipio, on = 'CO_MUN', how='left')

In [13]:
df_exportacao_mun_ncm = df_exportacao_mun_ncm.merge(df_urf, on='CO_URF', how='left' )

In [14]:
df_exportacao_mun_ncm = df_exportacao_mun_ncm.merge(df_via, on='CO_VIA', how='left')

In [15]:
# Check
# df_exportacao_mun_ncm[(df_exportacao_mun_ncm['CO_MUN'].notna()) & (df_exportacao_mun_ncm['NO_MUN'].isna())]

# Percentual de cobertura

In [20]:
df_exportacao_mun_ncm.notna().sum()/df_exportacao_mun_ncm.shape[0]

CO_NCM        1.000000
CO_MUN        0.331802
CO_VIA        1.000000
CO_URF        1.000000
NO_NCM_POR    1.000000
NO_NCM_ESP    1.000000
NO_NCM_ING    1.000000
NO_MUN        0.331802
NO_MUN_MIN    0.331802
SG_UF         0.331802
NO_URF        1.000000
NO_VIA        1.000000
dtype: float64

# Output data

In [18]:
df_exportacao_mun_ncm.to_excel('../data/02_output/exportacao.xlsx', index=False)

In [ ]:
df_exportacao_mun_ncm.to_csv('../data/02_output/exportacao.csv', index=False)